In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window as W

In [2]:
from pyspark.sql.types import DateType,StringType

In [3]:
spark = SparkSession.builder.appName("Test").getOrCreate()

In [4]:
student_data = [
    {"student_id": 1,"name":"Khan"},
     {"student_id": 2,"name":"Rahul"},
    {"student_id": 3,"name":"Lavan"},
]
schema = "student_id int,name string"
df = spark.createDataFrame(data = student_data,schema = schema)

In [5]:
df.show()

+----------+-----+
|student_id| name|
+----------+-----+
|         1| Khan|
|         2|Rahul|
|         3|Lavan|
+----------+-----+



In [6]:
student_marks = [
    {"student_id": 1,"subject":"Pyspark","marks":90},
    {"student_id": 1,"subject":"SQL","marks":100},
    {"student_id": 2,"subject":"SQL","marks":70},
    {"student_id": 2,"subject":"Pyspark","marks":60},
    {"student_id": 3,"subject":"SQL","marks":30},
    {"student_id": 3,"subject":"Pyspark","marks":20},
]

In [7]:
student_marks = spark.createDataFrame(student_marks)
student_marks.show()

+-----+----------+-------+
|marks|student_id|subject|
+-----+----------+-------+
|   90|         1|Pyspark|
|  100|         1|    SQL|
|   70|         2|    SQL|
|   60|         2|Pyspark|
|   30|         3|    SQL|
|   20|         3|Pyspark|
+-----+----------+-------+



In [9]:
df.alias('stu').join(student_marks.alias('marks'),F.col('stu.student_id') == F.col('marks.student_id'),'inner').show()

+----------+-----+-----+----------+-------+
|student_id| name|marks|student_id|subject|
+----------+-----+-----+----------+-------+
|         1| Khan|   90|         1|Pyspark|
|         1| Khan|  100|         1|    SQL|
|         2|Rahul|   70|         2|    SQL|
|         2|Rahul|   60|         2|Pyspark|
|         3|Lavan|   30|         3|    SQL|
|         3|Lavan|   20|         3|Pyspark|
+----------+-----+-----+----------+-------+



In [10]:
df.alias('stu').join(student_marks.alias('sub'),F.col('stu.student_id') == F.col('sub.student_id'),'inner').\
groupBy('stu.student_id').agg(
    F.sum(F.col('sub.marks')).alias('total_marks')
).show()

+----------+-----------+
|student_id|total_marks|
+----------+-----------+
|         1|        190|
|         3|         50|
|         2|        130|
+----------+-----------+



In [15]:
expr = F.when(F.col('per') > 90,'First').\
when(F.col('per').between(60,69),'second').\
when(F.col('per').between(50,59),'third').\
otherwise('faile')
marks_data = student_marks.groupBy('student_id').agg( F.sum(F.col('marks')).alias('total_marks')).\
withColumn('per',(F.col('total_marks')/2)).withColumn('division',expr).show()

+----------+-----------+----+--------+
|student_id|total_marks| per|division|
+----------+-----------+----+--------+
|         1|        190|95.0|   First|
|         2|        130|65.0|  second|
|         3|         50|25.0|    fial|
+----------+-----------+----+--------+



In [ ]:
df.alias('stu').join(marks_data.alias('mar'),F.col('stu.student_id') == )